In [1]:
# note: old version of sklearn in conda environment

import sklearn
sklearn.__version__

'0.23.0'

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import time
from sklearn.ensemble import GradientBoostingClassifier
import joblib
# import statsmodels.api as sm
# import statsmodels.formula.api as smf
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

# first one here is from https://github.com/meysubb/cfbscrapR-MISC/blob/master/EPA_WPA/02-EPA-Model.R#L245
reg_equation_cfb = 'next_drive_point ~ time_remaining + adjusted_yardline + C(down) + log_distance + goal_to_go + under_two + log_distance*C(down) + adjusted_yardline*C(down) + goal_to_go*log_distance'
reg_equation_spl = 'drive_point ~ C(down) + distance + adjusted_yardline + C(down):distance + C(down):adjusted_yardline + period + margin'
reg_equation_ake = 'drive_point ~ C(down) + distance + adjusted_yardline + C(down):distance + C(down):adjusted_yardline + period + margin + time_remaining'
reg_equation = reg_equation_cfb




In [3]:
drive_data = pd.DataFrame()
game_data = pd.DataFrame()
play_data = pd.DataFrame()

# downloaded files from collegefootballdata.com
def retrieveCfbDataFile(endpoint, year):
    return pd.read_csv(f"data/{endpoint}/{year}.csv", encoding='latin-1')

for i in range(2012, 2020):
    drive = retrieveCfbDataFile('drives',i)
    drive['year'] = i
    drive_data = drive_data.append(drive, sort=False)
    
    gm = retrieveCfbDataFile('games',i)
    gm['year'] = i
    game_data = game_data.append(gm, sort=False)
    
    plys = retrieveCfbDataFile('pbp',i)
    plys['year'] = i
    plys["play_id"] = plys.id.astype(int)
    plys.id = plys.id.astype(int)
    play_data = play_data.append(plys, sort=False)

print(f"Total Games: {len(game_data)}")
print(f"Total Drives: {len(drive_data)}")
print(f"Total Plays: {len(play_data)}")

Total Games: 6644
Total Drives: 171692
Total Plays: 1210147


In [4]:
# drop FCS games 
play_data = play_data[(play_data['offense_conference'].notna()) & (play_data['defense_conference'].notna())]
print(f"Total FBS Plays: {len(play_data)}")

Total FBS Plays: 1055257


In [5]:
game_data['game_id'] = game_data['id']
data = pd.merge(drive_data,game_data,on='game_id')
data['drive_id'] = data['id_x']

In [6]:
pbp_data = pd.merge(play_data,data[['home_team','drive_id']],how='left',on='drive_id')

In [7]:
pbp_data['coef'] = (pbp_data['home_team'] == pbp_data['defense']).astype(int)
pbp_data['adjusted_yardline'] = 100*(1-pbp_data['coef']) +  (2*pbp_data['coef']-1)*pbp_data['yard_line'] #yard_line is defined by home team in API
pbp_data['margin'] = pbp_data['offense_score'] - pbp_data['defense_score']
pbp_data.loc[pbp_data.down > 4, 'down'] = 4
pbp_data.loc[pbp_data.period > 4, 'period'] = 5
pbp_data = pbp_data.sort_values(by="play_id", ascending=True)

In [8]:
# from cfbscrapR
pbp_data["clock.minutes"] = pbp_data.apply(lambda x: 15 + x["clock.minutes"] if (x.period in [1,3]) else x["clock.minutes"], axis=1)
pbp_data["time_remaining"] = pbp_data.apply(lambda x: (60 * x["clock.minutes"] + x["clock.seconds"]) if (x.period <= 4) else 0, axis=1)
pbp_data["log_distance"] = pbp_data.distance.apply(lambda x: np.log(0.5) if x == 0 else np.log(x))
pbp_data["goal_to_go"] = pbp_data.apply(lambda x: (x.distance >= (x.adjusted_yardline - 17)) if ("Field Goal" in x.play_type) else (x.distance >= x.adjusted_yardline), axis=1)
pbp_data["under_two"] = pbp_data.time_remaining.apply(lambda x: x <= 120)

pbp_data["abs_diff"] = abs(pbp_data.offense_score - pbp_data.defense_score)
pbp_data["ScoreDiff_W"] = (max(pbp_data.abs_diff) - pbp_data.abs_diff) / (max(pbp_data.abs_diff) - min(pbp_data.abs_diff))



In [9]:
data['drive_point'] = data.drive_result.apply(lambda x: 7 if (x == 'TD' or x == 'PUNT TD' or x == 'RUSHING TD' or x == 'PASSING TD') else (3 if (x == 'FG' or x == 'FG GOOD') else (-2 if x == 'SF' else -7 if ( x ==   'PUNT RETURN TD' or x == 'MISSED FG TD' or x == 'INT TD' or x == 'FUMBLE RETURN TD' or x == 'FUMBLE TD' or x == 'DOWNS TD' or x == 'INT RETURN TOUCH'  or x == 'FG MISSED TD' or x =='PUNT TD' or x == 'TURNOVER ON DOWNS TD' or x == 'DOWNS TD') else 0 )))

In [10]:
print(data.columns)

Index(['offense', 'offense_conference', 'defense', 'defense_conference',
       'game_id', 'id_x', 'scoring', 'start_period', 'start_yardline',
       'start_time.minutes', 'start_time.seconds', 'end_period',
       'end_yardline', 'end_time.minutes', 'end_time.seconds',
       'elapsed.minutes', 'elapsed.seconds', 'plays', 'yards', 'drive_result',
       'year_x', 'id_y', 'season', 'week', 'season_type', 'start_date',
       'neutral_site', 'conference_game', 'attendance', 'venue_id', 'venue',
       'home_team', 'home_conference', 'home_points', 'home_line_scores[0]',
       'home_line_scores[1]', 'home_line_scores[2]', 'home_line_scores[3]',
       'home_post_win_prob', 'away_team', 'away_conference', 'away_points',
       'away_line_scores[0]', 'away_line_scores[1]', 'away_line_scores[2]',
       'away_line_scores[3]', 'away_post_win_prob', 'year_y', 'drive_id',
       'drive_point'],
      dtype='object')


In [11]:
def find_game_next_score_half(drive_df):
    drive_df.drive_id = drive_df.drive_id.astype(int)
    drive_df = drive_df.sort_values(by="drive_id", ascending=True).reset_index()
    score_plays = drive_df[
        (drive_df.scoring == True)
        & (~drive_df.drive_result.str.contains("END OF"))
    ].drop_duplicates('drive_id').index.to_series()
    
    final_df = pd.DataFrame()
    for x in range(0, len(drive_df)):
        tmp_df = find_next_score(x, score_plays, drive_df)
        final_df = final_df.append(tmp_df)
#     print(final_df)
    
    final_df2 = pd.merge(drive_df, final_df, left_index=True, right_index=True)
    final_df2['Drive_Score_Dist'] = final_df2.DSH - final_df2.drive_id.astype(int)
    final_df2['Drive_Score_Dist_W'] = (max(final_df2.Drive_Score_Dist) - final_df2.Drive_Score_Dist) / (max(final_df2.Drive_Score_Dist) - min(final_df2.Drive_Score_Dist))
    return final_df2

def find_next_score(play_i, score_plays, dat_drive):
    defense_tds = ["FUMBLE RETURN TD", "FUMBLE TD", "INT RETURN TOUCH", ""]
    next_score_i = None
    try:
        next_score_i = score_plays[
            score_plays >= play_i
        ].iloc[0]
    except IndexError:
        next_score_i = None
        
#     print(f"Checking play index: {play_i}")
#     print(f"Checking next score index: {next_score_i}")
    try:
        test = dat_drive.iloc[play_i]
    except IndexError:
        print(f"play_i {play_i} out of bounds in dat_drive (size: {len(dat_drive)})")
        
    if (next_score_i != None):
        try:
            test = dat_drive.iloc[next_score_i]
        except IndexError:
            print(f"next_score_i {next_score_i} out of bounds in dat_drive (size: {len(dat_drive)})")
        
        
    if ((next_score_i == None) 
        or (dat_drive.iloc[play_i].start_period <= 2 and dat_drive.iloc[next_score_i].start_period in [3,4])
       or (dat_drive.iloc[play_i].start_period in [3,4] and dat_drive.iloc[next_score_i].start_period > 4)):
        score_drive = dat_drive.iloc[play_i].drive_id
        next_score = 0
        return pd.DataFrame({"NSH" : next_score, "DSH" : score_drive}, index=[play_i])
    else:
        next_score = 0
        score_drive = dat_drive.iloc[next_score_i].drive_id
        current_team = dat_drive.iloc[play_i].offense
        
        next_score_team = dat_drive.iloc[next_score_i].offense
        if (dat_drive.iloc[next_score_i].drive_result in defense_tds):
            next_score_team = dat_drive.iloc[next_score_i].defense
        
        if ("RETURN TD" in dat_drive.iloc[next_score_i].drive_result):
            if (current_team == next_score_team):
                next_score = -1 * dat_drive.iloc[next_score_i].drive_point
            else:
                next_score = dat_drive.iloc[next_score_i].drive_point
        else:
            if (current_team == next_score_team):
                next_score = dat_drive.iloc[next_score_i].drive_point
            else:
                next_score = -1 * dat_drive.iloc[next_score_i].drive_point
        return pd.DataFrame({"NSH" : next_score, "DSH" : score_drive}, index=[play_i])
    
data = find_game_next_score_half(data)
# data19

In [12]:
print(len(data))
print(len(data[data.Drive_Score_Dist_W < 1.0]))
print(len(data[data.NSH == -3]))

171692
97359
17557


In [13]:
# data.loc[data.drive_point == 0, 'drive_point'] = data['NSH']

In [14]:
drive_stuff = data[['drive_id','game_id','DSH','drive_point','drive_result','NSH','Drive_Score_Dist_W']]
drive_stuff.columns = ['drive_id','game_id','DSH','drive_point','drive_result','next_drive_point','Drive_Score_Dist_W']

pbp_data = pd.merge(pbp_data, drive_stuff, left_on='drive_id',right_on='drive_id')
pbp_data['Total_W'] = pbp_data.Drive_Score_Dist_W + pbp_data.ScoreDiff_W
pbp_data['Total_W_Scaled'] = (pbp_data.Total_W - min(pbp_data.Total_W)) / (max(pbp_data.Total_W) - min(pbp_data.Total_W))
pbp_data.head()

,id,offense,offense_conference,defense,defense_conference,home,away,offense_score,defense_score,drive_id,period,clock.minutes,clock.seconds,yard_line,down,distance,yards_gained,play_type,play_text,ppa,year,play_id,home_team,coef,adjusted_yardline,margin,time_remaining,log_distance,goal_to_go,under_two,abs_diff,ScoreDiff_W,game_id,DSH,drive_point,drive_result,next_drive_point,Drive_Score_Dist_W,Total_W,Total_W_Scaled
0,4005478320,Wake Forest,ACC,Army,FBS Independents,Wake Forest,Army,14,21,40054783210,2,2,14,39,2,10,15,Penalty,"ARMY Penalty, personal foul (N/A) to the WFrst...",NaN,2014,4005478320,Wake Forest,0,61,-7,134,2.302585,False,False,7,0.915663,400547832,40054783210,0,INT,0,1.0,1.915663,0.936364
1,400547832102977603,Army,FBS Independents,Wake Forest,ACC,Wake Forest,Army,21,14,40054783210,2,2,23,65,1,0,38,Kickoff,"Daniel Grochowski kickoff for 64 yds , Tyler H...",NaN,2014,400547832102977603,Wake Forest,1,65,7,143,-0.693147,False,False,7,0.915663,400547832,40054783210,0,INT,0,1.0,1.915663,0.936364
2,400547832102978501,Wake Forest,ACC,Army,FBS Independents,Wake Forest,Army,14,21,40054783210,2,2,14,39,1,10,0,Pass Incompletion,John Wolford pass incomplete to E.J. Scott,-1.107768,2014,400547832102978501,Wake Forest,0,61,-7,134,2.302585,False,False,7,0.915663,400547832,40054783210,0,INT,0,1.0,1.915663,0.936364
3,4005478910,Texas,Big 12,Oklahoma State,Big 12,Oklahoma State,Texas,13,0,4005478915,1,23,7,65,1,65,10,Penalty,"OKLAHOMA ST Penalty, illegal block (10 Yards) ...",NaN,2014,4005478910,Oklahoma State,1,65,13,1387,4.174387,True,False,13,0.843373,400547891,4005478916,0,PUNT,-3,1.0,1.843373,0.881818
4,400547891101976604,Oklahoma State,Big 12,Texas,Big 12,Oklahoma State,Texas,0,13,4005478915,1,17,33,9,1,10,21,Pass Reception,Daxx Garman pass complete to Brandon Sheperd f...,1.129399,2014,400547891101976604,Oklahoma State,0,91,-13,1053,2.302585,False,False,13,0.843373,400547891,4005478916,0,PUNT,-3,1.0,1.843373,0.881818


In [15]:
exclude_playtype = [
#     'Kickoff',  
#     'End Period',
#     'Kickoff Return (Offense)',
#     'Kickoff Return Touchdown', 
#     'End of Half', 
#     'Defensive 2pt Conversion',
#     'Uncategorized', 
#     'End of Game', 
#     'Timeout',
#     'placeholder'
    "Extra Point Missed",
    "Extra Point Good",
    "Timeout",
    "End of Half",
    "End of Game",
    "Uncategorized",
    #"Kickoff",
    "Penalty"
    #"Kickoff Return (Offense)",
    #"Kickoff Return Touchdown"
]

# game_end_drive = ['END OF HALF', 'END OF GAME', 'Uncategorized','END OF 4TH QUARTER', 'DOWNS TD','POSSESSION (FOR OT DRIVES)',"END OF 4TH QUARTER"]

regression_df = pbp_data[
    (pbp_data.down > 0)
    & (pbp_data.year >= 2014)
    & ~(pbp_data.play_type.isin(exclude_playtype)) 
    & (pbp_data.down.notna())
    & (pbp_data.time_remaining.notna())
    & (pbp_data.game_id.notna())
    & (pbp_data.log_distance.notna() & pbp_data.log_distance != -float('inf'))
    & (pbp_data.game_id != 400603838)
].dropna()

In [16]:
from patsy import dmatrices
# int_conv = regression_df.astype({"drive_point":int, "down": int, "distance":int, "adjusted_yardline": int, "period": int, "margin": int, "time_remaining": int})
y, X = dmatrices(reg_equation, regression_df, return_type='dataframe')
X.columns

Index(['Intercept', 'C(down)[T.2]', 'C(down)[T.3]', 'C(down)[T.4]',
       'goal_to_go[T.True]', 'under_two[T.True]', 'time_remaining',
       'adjusted_yardline', 'adjusted_yardline:C(down)[T.2]',
       'adjusted_yardline:C(down)[T.3]', 'adjusted_yardline:C(down)[T.4]',
       'log_distance', 'log_distance:C(down)[T.2]',
       'log_distance:C(down)[T.3]', 'log_distance:C(down)[T.4]',
       'goal_to_go[T.True]:log_distance'],
      dtype='object')

In [17]:
# feature_cols = ["Intercept", "C(down)[T.2]", "C(down)[T.3]", "C(down)[T.4]", "distance", "C(down)[T.2]:distance", "C(down)[T.3]:distance", "C(down)[T.4]:distance", "adjusted_yardline", "C(down)[T.2]:adjusted_yardline", "C(down)[T.3]:adjusted_yardline", "C(down)[T.4]:adjusted_yardline", "period", "margin"]
for c in X.columns:
    X[c] = X[c].astype(int)
y.next_drive_point = y.next_drive_point.astype(int)


In [18]:
X.columns = ['Intercept', 'down2', 'down3', 'down4',
       'goal_to_go', 'under_two', 'time_remaining',
       'adjusted_yardline', 'adjusted_yardline_down_2',
       'adjusted_yardline_down_3', 'adjusted_yardline_down_4',
       'log_distance', 'log_distance_down_2',
       'log_distance_down_3', 'log_distance_down_4',
       'goal_to_go_log_distance']

In [19]:
print("Base drive point\n",y.next_drive_point.value_counts())

Base drive point
  7    125425
-7     78990
 3     46858
-3     27196
 0     15874
 2       929
-2       569
Name: next_drive_point, dtype: int64


In [22]:
class_to_score_mapping = {
    0: 7,
    1: -7,
    2: 3,
    3: -3,
    4: 0,
    5: 2,
    6: -2
}
score_to_class_mapping = {
    7: 0,
    -7: 1,
    3: 2,
    -3: 3,
    0: 4,
    2: 5,
    -2: 6
}
y.next_drive_point = y.next_drive_point.apply(lambda x: score_to_class_mapping[x])
print("Mapped drive point\n",y.next_drive_point.value_counts())

Mapped drive point
 0    125425
1     78990
2     46858
3     27196
4     15874
5       929
6       569
Name: next_drive_point, dtype: int64


In [24]:
nrounds = 300
params = {
    'objective': 'multi:softprob',
    'booster' : 'gbtree',
    'eval_metric' : ['logloss'],
    "num_class" : y.next_drive_point.nunique(),
    "eta": 0.025,
    "gamma": 1,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "max_depth": 5,
    "min_child_weight": 1
}

dtrain = xgb.DMatrix(X, weight=regression_df.Total_W_Scaled, label=y.next_drive_point)

xgb_model = xgb.train(params, dtrain, num_boost_round=nrounds, verbose_eval=2)
# save for debug
xgb_model.dump_model('xgb_dump.json', with_stats=True, dump_format='json')

In [25]:
special_team_play_type = ['Kickoff','Punt','Kickoff Return (Offense)', 'Kickoff Return Touchdown','Field Goal Good', 'Field Goal Missed', 'Blocked Field Goal',
                          'Blocked Punt','Punt Return Touchdown','Blocked Punt Touchdown','Missed Field Goal Return','Uncategorized', 'Missed Field Goal Return Touchdown','Defensive 2pt Conversion']
timing_play_type = ['End Period','End of Game','Timeout','End of Half',"END OF GAME"]
turnover_play_type = ['Fumble Recovery (Opponent)','Pass Interception Return','Interception Return Touchdown','Fumble Return Touchdown','Safety','Interception','Pass Interception','Punt',
                     'Field Goal Missed', 'Blocked Field Goal', 'Blocked Punt','Punt Return Touchdown','Blocked Punt Touchdown','Missed Field Goal Return', 'Missed Field Goal Return Touchdown']
regular_play_type = ['Pass', 'Rush', 'Sack', 'Pass Reception', 'Passing Touchdown','Pass Incompletion', 'Fumble Recovery (Own)','Rushing Touchdown','Pass Interception','Pass Completion']
time_play = ['End Period','Timeout','End of Half','End of Game',"END OF GAME"]
PAT_miss_type= [ 'PAT MISSED','PAT failed', 'PAT blocked', 'PAT BLOCKED']
off_TD = ['Passing Touchdown','Rushing Touchdown']
def_TD = ['Interception Return Touchdown','Fumble Return Touchdown', 'Missed Field Goal Return Touchdown','Blocked Punt Touchdown','Punt Return Touchdown']

In [26]:
regular_play = pbp_data[~pbp_data.play_type.str.contains('Kickoff') & ~(pbp_data.play_type.isin(time_play)) &(pbp_data.down > 0) & (pbp_data.distance > 0)]
# regular_play.to_csv("./regular_pbp.csv", index=False,encoding="utf8")
print(len(regular_play.Total_W_Scaled))
regular_play.head()

925438


,id,offense,offense_conference,defense,defense_conference,home,away,offense_score,defense_score,drive_id,period,clock.minutes,clock.seconds,yard_line,down,distance,yards_gained,play_type,play_text,ppa,year,play_id,home_team,coef,adjusted_yardline,margin,time_remaining,log_distance,goal_to_go,under_two,abs_diff,ScoreDiff_W,game_id,DSH,drive_point,drive_result,next_drive_point,Drive_Score_Dist_W,Total_W,Total_W_Scaled
0,4005478320,Wake Forest,ACC,Army,FBS Independents,Wake Forest,Army,14,21,40054783210,2,2,14,39,2,10,15,Penalty,"ARMY Penalty, personal foul (N/A) to the WFrst...",NaN,2014,4005478320,Wake Forest,0,61,-7,134,2.302585,False,False,7,0.915663,400547832,40054783210,0,INT,0,1.0,1.915663,0.936364
2,400547832102978501,Wake Forest,ACC,Army,FBS Independents,Wake Forest,Army,14,21,40054783210,2,2,14,39,1,10,0,Pass Incompletion,John Wolford pass incomplete to E.J. Scott,-1.107768,2014,400547832102978501,Wake Forest,0,61,-7,134,2.302585,False,False,7,0.915663,400547832,40054783210,0,INT,0,1.0,1.915663,0.936364
3,4005478910,Texas,Big 12,Oklahoma State,Big 12,Oklahoma State,Texas,13,0,4005478915,1,23,7,65,1,65,10,Penalty,"OKLAHOMA ST Penalty, illegal block (10 Yards) ...",NaN,2014,4005478910,Oklahoma State,1,65,13,1387,4.174387,True,False,13,0.843373,400547891,4005478916,0,PUNT,-3,1.0,1.843373,0.881818
4,400547891101976604,Oklahoma State,Big 12,Texas,Big 12,Oklahoma State,Texas,0,13,4005478915,1,17,33,9,1,10,21,Pass Reception,Daxx Garman pass complete to Brandon Sheperd f...,1.129399,2014,400547891101976604,Oklahoma State,0,91,-13,1053,2.302585,False,False,13,0.843373,400547891,4005478916,0,PUNT,-3,1.0,1.843373,0.881818
5,400547891101976605,Oklahoma State,Big 12,Texas,Big 12,Oklahoma State,Texas,0,13,4005478915,1,17,33,30,1,10,-15,Penalty,"Hill, Tyreek rush for 5 yards to the OSU35 (Ha...",NaN,2014,400547891101976605,Oklahoma State,0,70,-13,1053,2.302585,False,False,13,0.843373,400547891,4005478916,0,PUNT,-3,1.0,1.843373,0.881818


In [27]:
CFB_teams_list = pd.read_csv('https://raw.githubusercontent.com/903124/CFB_EPA_data/master/.ipynb_checkpoints/cfb_teams_list-checkpoint.csv',encoding='utf-8')

In [28]:
CFB_teams_list.full_name.unique()

array(['Abilene Christian', 'Air Force', 'Akron', 'Alabama',
       'Alabama A&M', 'Albany', 'Alcorn State', 'Appalachian State',
       'Arizona', 'Arizona State', 'Arkansas', 'Arkansas State',
       'Arkansas-Pine Bluff', 'Army', 'Auburn', 'Austin Peay', 'BYU',
       'Ball State', 'Baylor', 'Bethune-Cookman', 'Boise State',
       'Boston College', 'Bowling Green', 'Buffalo', 'California',
       'Campbell', 'Central Arkansas', 'Central Connecticut',
       'Central Michigan', 'Charleston Southern', 'Charlotte',
       'Cincinnati', 'Clemson', 'Coastal Carolina', 'Colorado',
       'Colorado State', 'Connecticut', 'Delaware State', 'Drake', 'Duke',
       'Duquesne', 'East Carolina', 'Eastern Illinois',
       'Eastern Kentucky', 'Eastern Michigan', 'Eastern Washington',
       'Elon', 'Florida', 'Florida Atlantic', 'Florida International',
       'Florida State', 'Fordham', 'Fresno State', 'Gardner-Webb',
       'Georgia', 'Georgia Southern', 'Georgia State', 'Georgia Tech',
     

In [29]:
regular_play = pd.merge(regular_play,CFB_teams_list,left_on=['offense'],right_on=['full_name'])
regular_play.rename(columns={'abbreviation':'off_abbr', 'full_name': 'off_full_name'}, inplace=True)
regular_play = pd.merge(regular_play,CFB_teams_list,left_on=['defense'],right_on=['full_name'])
regular_play.rename(columns={'abbreviation':'def_abbr', 'full_name': 'def_full_name'}, inplace=True)

In [30]:
# there are some plays in prev seasons that have no drive_point set (and ended up being at the end of a game); this broke the produced dmatrices bc it drops NAs in the outcome var
# regular_play[regular_play.drive_point.isna() == True]
regular_play.loc[regular_play.drive_point.isna() == True, "drive_point"] = 0.0

invalid_types = ["Uncategorized", "placeholder"]

regular_play = regular_play[
    (regular_play.play_text.notna())
    & (regular_play.play_type.notna())
]
regular_play = regular_play[
    (regular_play.play_type != 'Penalty')
    & (regular_play.play_type != 'Interception') 
    & ~(regular_play.play_text.str.lower().str.contains('penalty')) 
    & ~(regular_play.play_type.str.lower().str.contains('uncategorized')) 
    & ~(regular_play.play_type.str.lower().str.contains('placeholder'))
]
regular_play.head()

,id,offense,offense_conference,defense,defense_conference,home,away,offense_score,defense_score,drive_id,period,clock.minutes,clock.seconds,yard_line,down,distance,yards_gained,play_type,play_text,ppa,year,play_id,home_team,coef,adjusted_yardline,margin,time_remaining,log_distance,goal_to_go,under_two,abs_diff,ScoreDiff_W,game_id,DSH,drive_point,drive_result,next_drive_point,Drive_Score_Dist_W,Total_W,Total_W_Scaled,off_full_name,off_abbr,def_full_name,def_abbr
1,400547832102978501,Wake Forest,ACC,Army,FBS Independents,Wake Forest,Army,14,21,40054783210,2,2,14,39,1,10,0,Pass Incompletion,John Wolford pass incomplete to E.J. Scott,-1.107768,2014,400547832102978501,Wake Forest,0,61,-7,134,2.302585,False,False,7,0.915663,400547832,40054783210,0,INT,0,1.0,1.915663,0.936364,Wake Forest,WFrst,Army,Army
2,322660154017,Wake Forest,ACC,Army,FBS Independents,Wake Forest,Army,0,6,32266015402,1,26,35,18,1,10,6,Pass Completion,Tanner Price pass complete to Michael Campanar...,0.271467,2012,322660154017,Wake Forest,0,82,-6,1595,2.302585,False,False,6,0.927711,322660154,32266015402,7,RUSHING TD,7,1.0,1.927711,0.945455,Wake Forest,WFrst,Army,Army
3,322660154018,Wake Forest,ACC,Army,FBS Independents,Wake Forest,Army,0,6,32266015402,1,25,56,24,2,4,0,Pass Incompletion,Tanner Price pass incomplete to Sherman Raglan...,-0.654563,2012,322660154018,Wake Forest,0,76,-6,1556,1.386294,False,False,6,0.927711,322660154,32266015402,7,RUSHING TD,7,1.0,1.927711,0.945455,Wake Forest,WFrst,Army,Army
4,322660154019,Wake Forest,ACC,Army,FBS Independents,Wake Forest,Army,0,6,32266015402,1,25,45,24,3,4,47,Pass Completion,Tanner Price pass complete to Brandon Terry fo...,4.083093,2012,322660154019,Wake Forest,0,76,-6,1545,1.386294,False,False,6,0.927711,322660154,32266015402,7,RUSHING TD,7,1.0,1.927711,0.945455,Wake Forest,WFrst,Army,Army
5,322660154020,Wake Forest,ACC,Army,FBS Independents,Wake Forest,Army,0,6,32266015402,1,25,25,71,1,10,10,Rush,Michael Campanaro rush for 10 yards to the Arm...,NaN,2012,322660154020,Wake Forest,0,29,-6,1525,2.302585,False,False,6,0.927711,322660154,32266015402,7,RUSHING TD,7,1.0,1.927711,0.945455,Wake Forest,WFrst,Army,Army


In [31]:
turnover_plays = [
    "Blocked Field Goal",
    "Blocked Field Goal Touchdown",    
    "Blocked Punt",
    "Blocked Punt Touchdown",
    "Field Goal Missed",
    "Missed Field Goal Return",
    "Missed Field Goal Return Touchdown",    
    "Fumble Recovery (Opponent)",
    "Fumble Recovery (Opponent) Touchdown",
    "Fumble Return Touchdown",
    "Fumble Return Touchdown Touchdown",
    "Defensive 2pt Conversion",
    "Pass Interception",
    "Interception",
    "Interception Return Touchdown",
    "Pass Interception Return",
    "Pass Interception Return Touchdown",
    "Punt",
    "Punt Return Touchdown",
    "Sack Touchdown",
    "Uncategorized Touchdown"
]

defense_score_vec = [
    "Blocked Punt Touchdown",
    "Blocked Field Goal Touchdown",
    "Missed Field Goal Return Touchdown",
    "Punt Return Touchdown",
    "Fumble Recovery (Opponent) Touchdown",    
    "Fumble Return Touchdown",
    "Fumble Return Touchdown Touchdown",
    "Defensive 2pt Conversion",
    "Safety",
    "Sack Touchdown",    
    "Interception Return Touchdown",
    "Pass Interception Return Touchdown",
    "Uncategorized Touchdown"
]

normal_play = [
    "Rush",
    "Pass",
    "Pass Completion",
    "Pass Reception",
    "Pass Incompletion",
    "Sack",
    "Fumble Recovery (Own)"
]

score = [
    "Passing Touchdown", 
    "Rushing Touchdown", 
    "Field Goal Good",
    "Pass Reception Touchdown",
    "Fumble Recovery (Own) Touchdown",
    "Punt Touchdown",
    "Rushing Touchdown Touchdown"         
]

kickoff = [
    "Kickoff",
    "Kickoff Return (Offense)",
    "Kickoff Return Touchdown",
    "Kickoff Touchdown"
]

def determine_new_down(row):
    if ((row.play_type in score) or (row.play_type in kickoff) or (row.play_type in turnover_plays) or (row.play_type in defense_score_vec)):
        return 1
    elif ((row.play_type in normal_play) & (row.yards_gained >= row.distance)):
        return 1
    elif ((row.play_type in normal_play) & (row.yards_gained < row.distance) & (row.down <= 3)):
        return row.down + 1
    elif ((row.play_type in normal_play) & (row.yards_gained < row.distance) & (row.down == 4)):
        return 1
    else:
        return None
    
def determine_new_distance(row):
    if ((row.play_type in normal_play) & (row.yards_gained >= row.distance) & (row.adjusted_yardline - row.yards_gained >= 10)):
        return 10
    elif ((row.play_type in normal_play) & (row.yards_gained >= row.distance) & (row.adjusted_yardline - row.yards_gained <= 10)):
        return row.adjusted_yardline
    elif ((row.play_type in normal_play) & (row.yards_gained < row.distance) & (row.down <= 3)):
        return row.distance - row.yards_gained
    elif ((row.play_type in normal_play) & (row.yards_gained < row.distance) & (row.down == 4) & (100 - (row.adjusted_yardline - row.yards_gained) > 10)):
        return 10
    elif ((row.play_type in normal_play) & (row.yards_gained < row.distance) & (row.down == 4) & (100 - (row.adjusted_yardline - row.yards_gained) <= 10)):
        return 100 - row.adjusted_yardline
    elif (row.play_type in turnover_plays):
        return 10
    elif (row.play_type in defense_score_vec):
        return 0
    elif (row.play_type in score):
        return 0
    elif (row.play_type in kickoff):
        return 10
    else:
        return None

def determine_new_yardline(row):
    if (row.play_type in normal_play):
        return row.adjusted_yardline - row.yards_gained
    elif (row.play_type in score):
        return 0
    elif (row.play_type in defense_score_vec):
        return 0
    elif (row.play_type in kickoff):
        return 75
    elif (row.play_type in turnover_plays):
        return 100 - row.adjusted_yardline + row.yards_gained
    else:
        return None

def determine_turnover_indicator(row):
    return 1 if ((row.play_type in turnover_plays) or (row.play_type in defense_score_vec) or ((row.play_type in normal_play) & row.yards_gained < row.distance & row.down == 4)) else 0

import re
def determine_punt_yardline(row):
    try:
        yds_punted = re.search('(?<=for)[^,]+ (\d+)', row.play_text).group(1).astype(float)
    except:
        yds_punted = 0
    return 100 - ((row.yard_line - yds_punted) if (row.yard_line > 50) else (row.yard_line + yds_punted))

def prep_end_vars(dat):
    print("starting basic setup of end vars")
    dat.play_id = dat.play_id.astype(int)
    dat.id = dat.id.astype(int)
    dat = dat.sort_values(by=["game_id","play_id"], ascending=True)
    dat['half'] = dat.period.apply(lambda x: 1 if (x <= 2) else 2)
    dat['new_yardline'] = 0
    dat['new_down'] = 0
    dat['new_distance'] = 0
    dat['turnover_end'] = False
    dat['next_offense'] = dat.offense.shift(-1)

    dat.loc[((dat.play_type.isin(turnover_plays)) | (dat.next_offense != dat.offense)), "turnover_end"] = True
    dat['turnover_indicator'] = dat.apply(lambda x: determine_turnover_indicator(x), axis = 1)
    dat.loc[dat.play_type.str.contains("Kickoff"), 'down'] = 5
    dat['new_down'] = dat.apply(lambda row: determine_new_down(row), axis=1)
    dat['new_distance'] = dat.apply(lambda row: determine_new_distance(row), axis=1)
    dat['new_yardline'] = dat.apply(lambda row: determine_new_yardline(row), axis=1)
    dat['new_time_remaining'] = dat.time_remaining.shift(-1)
    dat.loc[dat.new_time_remaining.isna(), 'new_time_remaining'] = 0
    dat['new_log_distance'] = dat.new_distance.apply(lambda x: np.log(0.5) if x == 0 else np.log(x)) #np.log(0.5) if (dat.new_distance == 0) else np.log(dat.new_distance)
    dat['new_goal_to_go'] = (dat.new_yardline <= dat.new_distance)
    dat['new_under_two'] = (dat.new_time_remaining <= 120)
    dat["end_half_game"] = False
    print("done with basic setup")
    
    print("Updating punt stuff")
#     is_punt = dat.play_type.str.contains("Punt")
#     is_punt_touchback = is_punt & (dat.play_text.str.contains("touchback"))
#     yds_gained_more_0 = (dat.yards_gained > 0) & is_punt
    dat.loc[dat.play_type.str.contains("Punt"), "new_down"] = 1
    dat.loc[dat.play_type.str.contains("Punt"), "new_distance"] = 10
    dat.loc[dat.play_type.str.contains("Punt"), "new_log_distance"] = np.log(10)
    dat.loc[dat.play_type.str.contains("Punt"), "new_goal_to_go"] = False
    dat.loc[(dat.play_type.str.contains("Punt") & (dat.play_text.str.contains("touchback"))), "new_yardline"] = 80
    dat.loc[(dat.play_type.str.contains("Punt") & (dat.yards_gained > 0)), "new_yardline"] = 100 - (dat.adjusted_yardline - dat.yards_gained)
    
    dat.loc[(dat.yards_gained == 0) & (dat.play_type.str.contains("Punt")) & ~((dat.play_type.str.contains("Punt")) & (dat.play_text.str.contains("touchback"))), "new_yardline"] = dat.apply(lambda x: determine_punt_yardline(x), axis=1)

    print("Updating end of half stuff")
#     end_of_half_plays = (dat.new_time_remaining == 0)
#     if (end_of_half_plays.any()):
    dat.loc[(dat.new_time_remaining == 0), "new_yardline"] = 99
    dat.loc[(dat.new_time_remaining == 0), "new_down"] = 4
    dat.loc[(dat.new_time_remaining == 0), "new_distance"] = 99
    dat.loc[(dat.new_time_remaining == 0), "end_half_game"] = 1
    dat.loc[(dat.new_time_remaining == 0), "new_log_distance"] = np.log(99)
#         dat[end_of_half_plays, "new_goal_to_go"] = False
    dat.loc[(dat.new_time_remaining == 0), "new_under_two"] = (dat.new_time_remaining <= 120)
    dat.loc[(dat.new_time_remaining == 0), "end_half_game"] = True
    
    print("Fixing misc stuff with bad values")
#     na_yd_line = dat[(dat.new_yardline.isna()) or (dat.new_yardline >= 100)]
    dat.loc[(dat.new_yardline.isna()) | (dat.new_yardline >= 100), "new_yardline"] = dat.new_yardline.shift(-1)
    
#     neg_distance = dat[(dat.new_distance < 0)]
    dat.loc[(dat.new_distance < 0), "new_distance"] = dat.new_distance.shift(-1)
    dat.loc[(dat.new_distance < 0), "new_log_distance"] = dat.new_distance.apply(lambda x: np.log(0.5) if x == 0 else np.log(x))
    
#     missing_yd_line = (dat.new_yardline == 0) or (dat.new_yardline.isna())
    dat.loc[(dat.new_yardline == 0) | (dat.new_yardline.isna()), "new_yardline"] = 99
    dat.loc[(dat.new_yardline == 0) | (dat.new_yardline.isna()), "new_log_distance"] = np.log(99)
    
    print("Done!")
    return dat
#     return dat[["play_id","game_id", "drive_id", "new_time_remaining","new_down","new_distance","new_yardline","new_log_distance","new_goal_to_go","new_under_two","end_half_game","turnover","Total_W_Scaled"]]    

In [32]:
regular_play = prep_end_vars(regular_play)
regular_play #= pd.merge(regular_play, base_end, left_on="play_id", right_on="play_id")

starting basic setup of end vars
done with basic setup
Updating punt stuff
Updating end of half stuff
Fixing misc stuff with bad values
Done!


,id,offense,offense_conference,defense,defense_conference,home,away,offense_score,defense_score,drive_id,period,clock.minutes,clock.seconds,yard_line,down,distance,yards_gained,play_type,play_text,ppa,year,play_id,home_team,coef,adjusted_yardline,margin,time_remaining,log_distance,goal_to_go,under_two,abs_diff,ScoreDiff_W,game_id,DSH,drive_point,drive_result,next_drive_point,Drive_Score_Dist_W,Total_W,Total_W_Scaled,off_full_name,off_abbr,def_full_name,def_abbr,half,new_yardline,new_down,new_distance,turnover_end,next_offense,turnover_indicator,new_time_remaining,new_log_distance,new_goal_to_go,new_under_two,end_half_game
657094,322430041002,UMass,Mid-American,Connecticut,Big East,Connecticut,UMass,0,0,32243004101,1,29,45,80,1,10,1,Rush,Chris Burns rush for 1 yard to the UMass 21.,-0.468828,2012,322430041002,Connecticut,1,80,0,1785,2.302585,False,False,0,1.000000,322430041,32243004102,0,PUNT,-7,1.0,2.000000,1.000000,UMass,UMass,Connecticut,UConn,1,79.0,2,9,False,UMass,0,1760.0,2.197225,False,False,False
657095,322430041003,UMass,Mid-American,Connecticut,Big East,Connecticut,UMass,0,0,32243004101,1,29,20,79,2,9,0,Pass Incompletion,Mike Wegzyn pass incomplete.,-0.310093,2012,322430041003,Connecticut,1,79,0,1760,2.197225,False,False,0,1.000000,322430041,32243004102,0,PUNT,-7,1.0,2.000000,1.000000,UMass,UMass,Connecticut,UConn,1,79.0,3,9,False,UMass,0,1750.0,2.197225,False,False,False
657096,322430041005,UMass,Mid-American,Connecticut,Big East,Connecticut,UMass,0,0,32243004101,1,29,10,79,3,9,5,Pass Completion,Mike Wegzyn pass complete to Marken Michel for...,0.019784,2012,322430041005,Connecticut,1,79,0,1750,2.197225,False,False,0,1.000000,322430041,32243004102,0,PUNT,-7,1.0,2.000000,1.000000,UMass,UMass,Connecticut,UConn,1,74.0,4,4,False,UMass,0,1708.0,1.386294,False,False,False
657097,322430041006,UMass,Mid-American,Connecticut,Big East,Connecticut,UMass,0,0,32243004101,1,28,28,74,4,4,-2,Punt,"Jeff Strait punt for 47 yards, returned by Nic...",NaN,2012,322430041006,Connecticut,1,74,0,1708,1.386294,False,False,0,1.000000,322430041,32243004102,0,PUNT,-7,1.0,2.000000,1.000000,UMass,UMass,Connecticut,UConn,1,24.0,1,10,True,Connecticut,1,1700.0,2.302585,False,False,False
570576,322430041008,Connecticut,Big East,UMass,Mid-American,Connecticut,UMass,0,0,32243004102,1,28,20,25,1,10,5,Rush,Lyle McCombs rush for 5 yards to the UConn 30.,0.094545,2012,322430041008,Connecticut,0,75,0,1700,2.302585,False,False,0,1.000000,322430041,32243004102,7,RUSHING TD,7,1.0,2.000000,1.000000,Connecticut,UConn,UMass,UMass,1,70.0,2,5,False,Connecticut,0,1666.0,1.609438,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540574,401183793104925609,New Mexico,Mountain West,Air Force,Mountain West,New Mexico,Air Force,16,44,40118379316,4,7,43,66,1,10,7,Rush,Chad Alexander run for 7 yds to the AFA 27,0.384582,2019,401183793104925609,New Mexico,0,34,-28,463,2.302585,False,False,28,0.662651,401183793,40118379316,7,TD,7,1.0,1.662651,0.745455,New Mexico,NMex,Air Force,AFA,2,27.0,2,3,False,New Mexico,0,117.0,1.098612,False,True,False
540575,401183793104984201,New Mexico,Mountain West,Air Force,Mountain West,New Mexico,Air Force,22,44,40118379316,4,1,57,73,2,3,27,Passing Touchdown,Trae Hall pass complete to Aaron Molina for 27...,2.552144,2019,401183793104984201,New Mexico,0,27,-22,117,1.098612,False,True,22,0.734940,401183793,40118379316,7,TD,7,1.0,1.734940,0.800000,New Mexico,NMex,Air Force,AFA,2,99.0,1,0,True,Air Force,0,117.0,-0.693147,True,True,False
182964,401183793104984204,Air Force,Mountain West,New Mexico,Mountain West,New Mexico,Air Force,44,22,40118379317,4,1,57,75,1,10,6,Rush,Mike Schmidt run for 6 yds to the AFA 31,0.333375,2019,401183793104984204,New Mexico,1,75,22,117,2.302585,False,True,22,0.734940,401183793,40118379317,0,END OF GAME,0,1.0,1.734940,0.800000,Ai

In [33]:
len(regular_play.Total_W_Scaled)

859583

In [34]:
# regular_play[(regular_play.yards_gained == 0) & (regular_play.play_type.str.contains("Punt")) & ~((regular_play.play_type.str.contains("Punt")) & (regular_play.play_text.str.contains("touchback")))].new_yardline = 100 if (punt_yd_line > 50) else 


In [35]:
y_test, X_test = dmatrices(reg_equation, regular_play, return_type='dataframe')
X_test.columns = ['Intercept', 'down2', 'down3', 'down4',
       'goal_to_go', 'under_two', 'time_remaining',
       'adjusted_yardline', 'adjusted_yardline_down_2',
       'adjusted_yardline_down_3', 'adjusted_yardline_down_4',
       'log_distance', 'log_distance_down_2',
       'log_distance_down_3', 'log_distance_down_4',
       'goal_to_go_log_distance']

dtest = xgb.DMatrix(X_test, weight=regular_play.Total_W_Scaled, label=y_test)
EP_predict = xgb_model.predict(dtest)
EP_predict

EP = EP_predict[:,0] * class_to_score_mapping[0] + EP_predict[:,1] * class_to_score_mapping[1] + EP_predict[:,2] * class_to_score_mapping[2] + EP_predict[:,3] * class_to_score_mapping[3] + EP_predict[:,4] * class_to_score_mapping[4]
# print(len(EP))
regular_play['EP_start'] = EP

In [36]:
out_df = pd.DataFrame({'play_type':regular_play['play_type'],'play_text':regular_play['play_text'],'yards_gained':regular_play['yards_gained'], 'down':regular_play['new_down'],'distance':regular_play['new_distance'],'adjusted_yardline':regular_play['new_yardline'],'margin':regular_play['margin'], 'period':regular_play['period'],'drive_point':regular_play['drive_point'],'next_drive_point':regular_play['next_drive_point'], 'log_distance':regular_play['new_log_distance'],'goal_to_go':regular_play['new_goal_to_go'],'time_remaining':regular_play['new_time_remaining'], 'under_two':regular_play['new_under_two'], "Total_W_Scaled" : regular_play.Total_W_Scaled})
out_df[out_df.down.isna()].play_type.value_counts()

Series([], Name: play_type, dtype: int64)

In [37]:
y_end, X_end = dmatrices(reg_equation, out_df, return_type='dataframe')

X_end.columns = ['Intercept', 'down2', 'down3', 'down4',
       'goal_to_go', 'under_two', 'time_remaining',
       'adjusted_yardline', 'adjusted_yardline_down_2',
       'adjusted_yardline_down_3', 'adjusted_yardline_down_4',
       'log_distance', 'log_distance_down_2',
       'log_distance_down_3', 'log_distance_down_4',
       'goal_to_go_log_distance']
len(X_end)

859583

In [38]:
dtest_end = xgb.DMatrix(X_end, weight=out_df.Total_W_Scaled, label=y_end)
EP_predict_end = xgb_model.predict(dtest_end)

In [39]:
EP_end = EP_predict_end[:,0] * class_to_score_mapping[0] + EP_predict_end[:,1] * class_to_score_mapping[1] + EP_predict_end[:,2] * class_to_score_mapping[2] + EP_predict_end[:,3] * class_to_score_mapping[3] + EP_predict_end[:,4] * class_to_score_mapping[4]

regular_play['EP_end'] = EP_end

In [40]:
# turnover_ps = (regular_play.turnover_end == True) and ~(regular_play.play_type == 'Kickoff')
regular_play.loc[(regular_play.turnover_end == True), "EP_end"] = -1 * regular_play.EP_end

regular_play.loc[(regular_play.end_half_game == True), "EP_end"] = 0

In [41]:
regular_play.loc[(regular_play.play_type.isin(off_TD)),'EP_end'] = 7
regular_play.loc[(regular_play.play_type.isin(def_TD)),'EP_end'] = -7
regular_play.loc[regular_play.play_type == 'Safety','EP_end'] = -2
regular_play.loc[(regular_play.play_type.isin(PAT_miss_type)),'EP_end'] = 6
regular_play.loc[regular_play.play_type == 'Field Goal Good','EP_end'] = 3

In [42]:
regular_play['EPA'] = regular_play['EP_end'] - regular_play['EP_start']

In [43]:
pass_play_type = ["Interception Return Touchdown","Pass Interception","Pass Incompletion","Pass Interception Return","Pass Reception","Pass Completion","Pass","Passing Touchdown","Sack"]
rush_play_type = ['Fumble Recovery (Opponent)','Fumble Recovery (Own)','Fumble Return Touchdown','Rush','Rushing Touchdown']

In [44]:
regular_play[regular_play.play_type.isin(pass_play_type)]['EPA'].mean()

0.10472939163446426

In [45]:
regular_play[regular_play.play_type.isin(rush_play_type)]['EPA'].mean()

-0.13040228188037872

In [46]:
regular_play.groupby('play_type')['EPA'].mean()

play_type
Blocked Field Goal                   -0.362948
Blocked Field Goal Touchdown          0.828288
Blocked Punt                         -0.749627
Blocked Punt Touchdown               -5.201476
Defensive 2pt Conversion             -4.790032
Field Goal Good                       2.925635
Field Goal Missed                    -0.225897
Fumble Recovery (Opponent)           -2.966336
Fumble Recovery (Own)                -0.921954
Fumble Return Touchdown              -7.153259
Interception Return Touchdown        -6.972321
Missed Field Goal Return              0.513615
Missed Field Goal Return Touchdown   -6.526301
Pass                                 -3.106970
Pass Completion                       0.535619
Pass Incompletion                    -0.908596
Pass Interception                    -2.154350
Pass Interception Return             -2.207031
Pass Reception                        0.881959
Passing Touchdown                     4.130691
Punt                                 -0.770362
Pun

In [47]:
regular_play[regular_play.play_type.isin(pass_play_type)]['ppa'].mean()

0.16998962783460378

In [48]:
regular_play[regular_play.play_type.isin(rush_play_type)]['ppa'].mean()

0.06823612722365852

In [49]:
# turnovers
regular_play[regular_play.play_type.isin(["Fumble Recovery (Opponent)","Pass Interception Return","Sack"])].groupby('play_type')['EPA'].mean()

play_type
Fumble Recovery (Opponent)   -2.966336
Pass Interception Return     -2.207031
Sack                         -1.156872
Name: EPA, dtype: float32

In [50]:
# passes
regular_play[(regular_play.play_type.isin(pass_play_type)) & (~regular_play.play_type.isin(["Fumble Recovery (Own)", "Interception Return Touchdown"]))].groupby('play_type')['EPA'].mean()


play_type
Pass                       -3.106970
Pass Completion             0.535619
Pass Incompletion          -0.908596
Pass Interception          -2.154350
Pass Interception Return   -2.207031
Pass Reception              0.881959
Passing Touchdown           4.130691
Sack                       -1.156872
Name: EPA, dtype: float32

In [51]:
# Rushes
regular_play[(regular_play.play_type.isin(rush_play_type)) & (~regular_play.play_type.isin(["Fumble Recovery (Own)", "Fumble Return Touchdown"]))].groupby('play_type')['EPA'].mean()


play_type
Fumble Recovery (Opponent)   -2.966336
Rush                         -0.199569
Rushing Touchdown             2.684442
Name: EPA, dtype: float32

In [52]:
# in_df = pd.DataFrame({'down':regular_play['new_down'],'distance':regular_play['new_distance'],'adjusted_yardline':regular_play['new_yardline'],'margin':regular_play['margin'], 'period':regular_play['period'],'drive_point':regular_play['drive_point'], 'log_distance':regular_play['log_distance'],'goal_to_go':regular_play['goal_to_go'],'time_remaining':regular_play['time_remaining'], 'under_two':regular_play['under_two'], 'yards_gained':regular_play['yards_gained']})
# y_in, X_in = dmatrices(reg_equation, in_df[(in_df.yards_gained == 4)], return_type='dataframe')
# X_test = X_in[(X_in["C(down)[T.2]"] == 0) & (X_in["C(down)[T.3]"] == 0) & (X_in["C(down)[T.4]"] == 0) & (X_in.distance == 10) & (X_in.adjusted_yardline == 75) & (X_in.period == 1) & (X_in.margin == 0)]
# X_test


In [53]:
# regular_play[(regular_play.play_type.isin(rush_play_type))].sort_values(by="EPA",ascending=True).head()

In [54]:
# in_df = pd.DataFrame({'down':regular_play['new_down'],'distance':regular_play['new_distance'],'adjusted_yardline':regular_play['new_yardline'],'margin':regular_play['margin'], 'period':regular_play['period'],'drive_point':regular_play['drive_point'], 'log_distance':regular_play['log_distance'],'goal_to_go':regular_play['goal_to_go'],'time_remaining':regular_play['time_remaining'], 'under_two':regular_play['under_two'], 'yards_gained':regular_play['yards_gained']})
# y_in, X_in = dmatrices(reg_equation, in_df, return_type='dataframe')
# # X_test = X_in #[(X_in["C(down)[T.2]"] == 1) & (X_in["C(down)[T.3]"] == 0) & (X_in["C(down)[T.4]"] == 0) & (X_in.distance == 3) & (X_in.adjusted_yardline == 5) & (X_in.period == 1) & (X_in.margin == 10)]
# # X_test


In [55]:
# def row_op_update(row):
#     row["down"] = 2
#     row["C(down)[T.2]"] = 1
#     row["distance"] = 6
#     row["adjusted_yardline"] = 71
#     row["C(down)[T.2]:distance"] = 6
#     row["C(down)[T.2]:adjusted_yardline"] = 71
#     row['drive_point'] = 0
#     return row

# output_df = pd.DataFrame(X_test).apply(lambda x: row_op_update(x), axis=1)
# output_df

In [56]:
# y_out, X_out = dmatrices(reg_equation, X_test, return_type='dataframe')
# X_out

In [57]:
# EP_start = clf.predict_proba(X_test)
# epa_start = EP_start[:,0]* -7 + EP_start[:,1] * -3 + EP_start[:,2] * -2 + EP_start[:,4] * 2 + EP_start[:,5] * 3 + EP_start[:,6] * 7

# EP_end = clf.predict_proba(output_df[["Intercept","C(down)[T.2]","C(down)[T.3]","C(down)[T.4]","distance","C(down)[T.2]:distance","C(down)[T.3]:distance","C(down)[T.4]:distance","adjusted_yardline","C(down)[T.2]:adjusted_yardline",	"C(down)[T.3]:adjusted_yardline","C(down)[T.4]:adjusted_yardline","period","margin"]])
# epa_end = 7#EP_end[:,0]* -7 + EP_end[:,1] * -3 + EP_end[:,2] * -2 + EP_end[:,4] * 2 + EP_end[:,5] * 3 + EP_end[:,6] * 7

# epa = epa_end - epa_start
# pd.DataFrame(data={"start":epa_start,"end":epa_end,"epa":epa})

In [58]:
regular_play[regular_play.play_type.str.contains("Touchdown") & (regular_play.yard_line == 95) & (regular_play.play_text.str.contains("Greene"))].head()

,id,offense,offense_conference,defense,defense_conference,home,away,offense_score,defense_score,drive_id,period,clock.minutes,clock.seconds,yard_line,down,distance,yards_gained,play_type,play_text,ppa,year,play_id,home_team,coef,adjusted_yardline,margin,time_remaining,log_distance,goal_to_go,under_two,abs_diff,ScoreDiff_W,game_id,DSH,drive_point,drive_result,next_drive_point,Drive_Score_Dist_W,Total_W,Total_W_Scaled,off_full_name,off_abbr,def_full_name,def_abbr,half,new_yardline,new_down,new_distance,turnover_end,next_offense,turnover_indicator,new_time_remaining,new_log_distance,new_goal_to_go,new_under_two,end_half_game,EP_start,EP_end,EPA
124604,400547917103965201,Bowling Green,Mid-American,Indiana,Big Ten,Bowling Green,Indiana,26,21,40054791721,3,18,47,95,2,5,5,Rushing Touchdown,"Travis Greene run for 5 yds for a TD, (Tyler T...",2.438578,2014,400547917103965201,Bowling Green,0,5,5,1127,1.609438,True,False,5,0.939759,400547917,40054791721,7,TD,7,1.0,1.939759,0.954545,Bowling Green,BwGrn,Indiana,Ind,2,99.0,1,0,True,Indiana,0,1113.0,-0.693147,True,False,False,4.811995,7.0,2.188005


In [59]:
# regular_play[(~regular_play.play_text.str.contains("penalty")) & (regular_play.down == 1) & (regular_play.yard_line == 25) & (regular_play.distance == 10) & (regular_play.yards_gained >= 10)].head()


In [60]:
# import coremltools

# y_end, X_end = dmatrices(reg_equation, out_df, return_type='dataframe')

# coreml_model = coremltools.converters.sklearn.convert(clf, X_end.columns.to_list(), "drive_point")
# coreml_model


In [61]:
# # CoreML model

import coremltools as cml
cml_model = cml.converters.xgboost.convert(xgb_model, force_32bit_float=False, mode="classifier", class_labels=[0,7,3,-7,-2], n_classes=len(class_to_score_mapping), feature_names=['Intercept', 'down2', 'down3', 'down4',
       'goal_to_go', 'under_two', 'time_remaining',
       'adjusted_yardline', 'adjusted_yardline_down_2',
       'adjusted_yardline_down_3', 'adjusted_yardline_down_4',
       'log_distance', 'log_distance_down_2',
       'log_distance_down_3', 'log_distance_down_4',
       'goal_to_go_log_distance'], target="drive_point")
cml_model.author = 'Saiem Gilani, Meyappan Subbiah, and Parker Fleming for R code; Parker Fleming for original Python model; Akshay Easwaran for updates.'
cml_model.license = 'MIT'
cml_model.short_description = 'Predicts expected points added by a football play given its context and activity. Translated from R, original model available as part of https://github.com/saiemgilani/cfbscrapR/.'


# Set feature descriptions manually
cml_model.input_description['Intercept'] = 'Constant.'
cml_model.input_description['down2'] = 'Signifies second down.'
cml_model.input_description['down3'] = 'Signifies third down.'
cml_model.input_description['down4'] = 'Signifies fourth down.'
cml_model.input_description['goal_to_go'] = 'Signifies that the number of yards to gain for a first down is fewer than the number of yards left to gain for a touchdown.'
cml_model.input_description['under_two'] = 'Notes that there are under two minutes remaining in the game.'
cml_model.input_description['time_remaining'] = 'The time remaining in the game.'
cml_model.input_description['adjusted_yardline'] = 'The yards left to gain towards the end zone.'
cml_model.input_description['adjusted_yardline_down_2'] = 'The yards left to gain towards the end zone on second down.'
cml_model.input_description['adjusted_yardline_down_3'] = 'The yards left to gain towards the end zone on third down.'
cml_model.input_description['adjusted_yardline_down_4'] = 'The yards left to gain towards the end zone on fourth down.'
cml_model.input_description['log_distance'] = 'The logarithm of the number of yards to gain a first down.'
cml_model.input_description['log_distance_down_2'] = 'The logarithm of the number of yards to gain a first down on second down.'
cml_model.input_description['log_distance_down_3'] = 'The logarithm of the number of yards to gain a first down on third down'
cml_model.input_description['log_distance_down_4'] = 'The logarithm of the number of yards to gain a first down on fourth down'
cml_model.input_description['goal_to_go_log_distance'] = 'The logarithm of the number of yards to gain a first down when in goal-to-go situations.'

# Set the output descriptions
cml_model.output_description['drive_point'] = 'The outcome of the drive, in points.'

# Save the model
cml_model.save('CFBEPA.mlmodel')
cml_model

/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/coremltools/models/model.py:119: RuntimeWarning: You will not be able to run predict() on this Core ML model. Underlying exception message was: Error compiling model: "Error reading protobuf spec. validator error: Error(s) in tree structure: 

  Specified output dimension (7) does not match the given number of classes (5).;".
  RuntimeWarning,


input {
  name: "Intercept"
  shortDescription: "Constant."
  type {
    doubleType {
    }
  }
}
input {
  name: "down2"
  shortDescription: "Signifies second down."
  type {
    doubleType {
    }
  }
}
input {
  name: "down3"
  shortDescription: "Signifies third down."
  type {
    doubleType {
    }
  }
}
input {
  name: "down4"
  shortDescription: "Signifies fourth down."
  type {
    doubleType {
    }
  }
}
input {
  name: "goal_to_go"
  shortDescription: "Signifies that the number of yards to gain for a first down is fewer than the number of yards left to gain for a touchdown."
  type {
    doubleType {
    }
  }
}
input {
  name: "under_two"
  shortDescription: "Notes that there are under two minutes remaining in the game."
  type {
    doubleType {
    }
  }
}
input {
  name: "time_remaining"
  shortDescription: "The time remaining in the game."
  type {
    doubleType {
    }
  }
}
input {
  name: "adjusted_yardline"
  shortDescription: "The yards left to gain towards the en

In [62]:
# pd.merge(out_df, X_end, left_index=True, right_index=True)

In [63]:
X_test.iloc[6311]

Intercept                     1.000000
down2                         0.000000
down3                         0.000000
down4                         0.000000
goal_to_go                    0.000000
under_two                     0.000000
time_remaining              160.000000
adjusted_yardline            75.000000
adjusted_yardline_down_2      0.000000
adjusted_yardline_down_3      0.000000
adjusted_yardline_down_4      0.000000
log_distance                  2.302585
log_distance_down_2           0.000000
log_distance_down_3           0.000000
log_distance_down_4           0.000000
goal_to_go_log_distance       0.000000
Name: 785837, dtype: float64

In [64]:
grouped_plays = regular_play[~((regular_play.play_text.str.contains("penalty")) | (regular_play.play_text.str.contains("PENALTY")))].groupby('play_type')
grouped_plays['EPA'].mean()

play_type
Blocked Field Goal                   -0.362948
Blocked Field Goal Touchdown          0.828288
Blocked Punt                         -0.749627
Blocked Punt Touchdown               -5.201476
Defensive 2pt Conversion             -4.790032
Field Goal Good                       2.925635
Field Goal Missed                    -0.225897
Fumble Recovery (Opponent)           -2.966336
Fumble Recovery (Own)                -0.921954
Fumble Return Touchdown              -7.153259
Interception Return Touchdown        -6.972321
Missed Field Goal Return              0.513615
Missed Field Goal Return Touchdown   -6.526301
Pass                                 -3.106970
Pass Completion                       0.535619
Pass Incompletion                    -0.908596
Pass Interception                    -2.154350
Pass Interception Return             -2.207031
Pass Reception                        0.881959
Passing Touchdown                     4.130691
Punt                                 -0.770362
Pun

In [65]:
# "Field Goal Good","Passing Touchdown","Pass Completion","Fumble Return Touchdown","Rushing Touchdown","Interception","Pass Incompletion","Rush"
# ignore: Punt,"Safety","Sack"
selected_groups = ["Passing Touchdown","Fumble Return Touchdown","Rushing Touchdown","Pass Completion","Field Goal Good","Pass Interception","Pass Incompletion","Rush","Field Goal Missed"]
composite_df = grouped_plays.apply(lambda x: x.sample(1)).reset_index(drop=True)
composite_df[composite_df.play_type.isin(selected_groups)]

,id,offense,offense_conference,defense,defense_conference,home,away,offense_score,defense_score,drive_id,period,clock.minutes,clock.seconds,yard_line,down,distance,yards_gained,play_type,play_text,ppa,year,play_id,home_team,coef,adjusted_yardline,margin,time_remaining,log_distance,goal_to_go,under_two,abs_diff,ScoreDiff_W,game_id,DSH,drive_point,drive_result,next_drive_point,Drive_Score_Dist_W,Total_W,Total_W_Scaled,off_full_name,off_abbr,def_full_name,def_abbr,half,new_yardline,new_down,new_distance,turnover_end,next_offense,turnover_indicator,new_time_remaining,new_log_distance,new_goal_to_go,new_under_two,end_half_game,EP_start,EP_end,EPA
5,400869835103969601,Appalachian State,Sun Belt,Georgia Southern,Sun Belt,Georgia Southern,Appalachian State,13,10,40086983519,3,18,3,11,4,5,28,Field Goal Good,Michael Rubino 28 yd FG GOOD,NaN,2016,400869835103969601,Georgia Southern,1,11,3,1083,1.609438,True,False,3,0.963855,400869835,40086983519,3,FG,3,1.0,1.963855,0.972727,Appalachian State,AppSt,Georgia Southern,GeoSo,2,99.0,1,0,True,Georgia Southern,1,1070.0,-0.693147,True,False,False,0.626730,3.000000,2.373270
6,400548271102947007,Colorado,Pac-12,California,Pac-12,California,Colorado,21,14,40054827111,2,5,29,21,4,10,39,Field Goal Missed,Will Oliver 39 Yard Field Goal Missed,NaN,2014,400548271102947007,California,1,21,7,329,2.302585,True,False,7,0.915663,400548271,40054827113,0,MISSED FG,7,1.0,1.915663,0.936364,Colorado,Colo,California,Cal,1,70.0,1,10,True,Colorado,1,257.0,2.302585,False,False,False,-1.147905,0.188917,1.336822
9,400869820105000285,New Mexico State,Sun Belt,Louisiana,Sun Belt,New Mexico State,Louisiana,31,31,40086982029,5,0,0,10,2,6,0,Fumble Return Touchdown,Darius Hoggins 0 Yd Fumble Return (Stevie Arti...,NaN,2016,400869820105000285,New Mexico State,0,90,0,0,1.791759,False,True,0,1.000000,400869820,40086982111,0,END OF HALF,-7,1.0,2.000000,1.000000,New Mexico State,NMxSt,Louisiana,LaLaf,2,99.0,4,99,True,Louisiana,1,0.0,4.595120,True,True,True,-1.841092,-7.000000,-5.158908
14,333340005047,UAB,Conference USA,Southern Mississippi,Conference USA,UAB,Southern Mississippi,7,7,33334000506,1,16,4,62,1,10,0,Pass Completion,Jonathan Perry pass complete to Darrin Reaves ...,-0.782892,2013,333340005047,UAB,0,38,0,964,2.302585,False,False,0,1.000000,333340005,33334000506,7,PASSING TD,7,1.0,2.000000,1.000000,UAB,UAB,Southern Mississippi,SoMis,1,38.0,2,10,False,UAB,0,923.0,2.302585,False,False,False,2.889477,2.069384,-0.820094
15,401012766101876201,Oregon,Pac-12,Washington,Pac-12,Oregon,Washington,0,0,4010127662,1,27,37,79,3,6,0,Pass Incompletion,Justin Herbert pass incomplete to Dillon Mitchell,NaN,2018,401012766101876201,Oregon,0,21,0,1657,1.791759,False,False,0,1.000000,401012766,4010127662,3,FG,3,1.0,2.000000,1.000000,Oregon,Oregn,Washington,Wash,1,21.0,4,6,False,Oregon,0,1649.0,1.791759,False,False,False,2.667827,0.086942,-2.580885
16,322730006010,Troy,Sun Belt,South Alabama,Sun Belt,South Alabama,Troy,0,0,32273000601,1,29,41,60,1,10,21,Pass Interception,Corey Robinson pass intercepted by B.J. Scott ...,-2.243099,2012,322730006010,South Alabama,1,60,0,1781,2.302585,False,False,0,1.000000,322730006,32273000604,0,INT,3,1.0,2.000000,1.000000,Troy,Troy,South Alabama,SAlab,1,61.0,1,10,True,South Alabama,1,1770.0,2.302585,False,False,False,1.953724,-1.750385,-3.704108
19,400869816102977501,Troy,Sun Belt,New Mexico State,Sun Belt,Troy,New Mexico State,28,0,40086981614,2,2,24,83,3,2,17,Passing Touchdown,Brandon Silvers pass complete to Sidney Davis ...,NaN,2016,400869816102977501,Troy,0,17,28,144,0.693147,False,False,28,0.662651,400869816,40086981614,7,TD,7,1.0,1.662651,0.745455,Troy,Troy,New Mexico State,NMxSt,1,99.0,1,0,True,New Mexico State,0,144.0,-0.693147,True,False,False,2.187159,7.000000,4.812841
22,323220026093,UCLA,Pac-12,USC,Pac-12,UCLA,USC,23,0,32322002608,2,7,38,84,1,10,16,Rush,Johnathan Franklin rush for 16 yards for a TOU...,NaN,2012,323220026093,UCLA,0,16,23,458,2.302585,False,False,23,0.722892,323220026,32322002608,7,

In [66]:
composite_df.iloc[15].play_text

'Justin Herbert pass incomplete to Dillon Mitchell'

In [67]:
# in_df = pd.DataFrame({'down':composite_df['new_down'],'distance':composite_df['new_distance'],'adjusted_yardline':composite_df['new_yardline'],'margin':composite_df['margin'], 'period':composite_df['period'],'drive_point':composite_df['drive_point'], 'log_distance':composite_df['log_distance'],'goal_to_go':composite_df['goal_to_go'],'time_remaining':composite_df['time_remaining'], 'under_two':composite_df['under_two'], 'yards_gained':composite_df['yards_gained']})
# y_in, X_in = dmatrices(reg_equation, in_df, return_type='dataframe')
# X_in.iloc[24]
# # this and all below was to prove that CoreML's borked when it comes to Sacks here

In [68]:
# EP_predict = clf.predict_proba(X_in)
# EP_test_end = EP_predict[:,0]* -7 + EP_predict[:,1] * -3 + EP_predict[:,2] * -2 + EP_predict[:,4] * 2 + EP_predict[:,5] * 3 + EP_predict[:,6] * 7
# print(EP_test_end[24])
# EP_predict[24]

In [69]:
# (-7 * 0.14209510369863743) + (-3 * 0.02542780623523315) + (-2 * 0.000068446669470519143) + (0 * 0.62909060112996429) + (2 * 0.0032402361821286113) + (3 * 0.017694614318773151) + (7 * 0.18238319176579296)
# # 

In [70]:
# (-7 * 0.132782240) + (-3 * 0.0248792017) + (-2 * 0.0000624865278) + (0 * 0.630871943) + (2 * 0.00273228324) + (3 * 0.0183370306) + (7 * 0.190334815)


In [71]:
# xcode_diff = pd.DataFrame({
#         "Intercept" : 1.0,
#         "C(down)[T.2]" : 0.0,
#         "C(down)[T.3]" : 0.0,
#         "C(down)[T.4]" : 1.0,
#         "distance" : 9.0,
#         "C(down)[T.2]:distance" : 0.0,
#         "C(down)[T.3]:distance" : 0.0,
#         "C(down)[T.4]:distance" : 9.0,
#         "adjusted_yardline" : 48.0,
#         "C(down)[T.2]:adjusted_yardline" : 0.0,
#         "C(down)[T.3]:adjusted_yardline" : 0.0,
#         "C(down)[T.4]:adjusted_yardline" : 48.0,
#         "period" : 4.0,
#         "margin" : -17.0
# }, index=[0])
# xcode_diff

In [72]:
# # X_in.iloc[24].equals(xcode_diff.iloc[0])
# pd.testing.assert_series_equal(X_in.iloc[24], xcode_diff.iloc[0], check_names=False)

In [73]:
# xcode_ep = clf.predict_proba(xcode_diff)
# xcode_ep_end = xcode_ep[:,0]* -7 + xcode_ep[:,1] * -3 + xcode_ep[:,2] * -2 + xcode_ep[:,4] * 2 + xcode_ep[:,5] * 3 + xcode_ep[:,6] * 7
# xcode_ep_end


In [74]:
# X_in.iloc[24].to_dict()

In [75]:
regular_play[regular_play.next_drive_point == -3].head()

,id,offense,offense_conference,defense,defense_conference,home,away,offense_score,defense_score,drive_id,period,clock.minutes,clock.seconds,yard_line,down,distance,yards_gained,play_type,play_text,ppa,year,play_id,home_team,coef,adjusted_yardline,margin,time_remaining,log_distance,goal_to_go,under_two,abs_diff,ScoreDiff_W,game_id,DSH,drive_point,drive_result,next_drive_point,Drive_Score_Dist_W,Total_W,Total_W_Scaled,off_full_name,off_abbr,def_full_name,def_abbr,half,new_yardline,new_down,new_distance,turnover_end,next_offense,turnover_indicator,new_time_remaining,new_log_distance,new_goal_to_go,new_under_two,end_half_game,EP_start,EP_end,EPA
657098,322430041033,UMass,Mid-American,Connecticut,Big East,Connecticut,UMass,0,7,32243004103,1,20,40,79,1,10,9,Pass Completion,Mike Wegzyn pass complete to Rob Blanchflower ...,1.142260,2012,322430041033,Connecticut,1,79,-7,1240,2.302585,False,False,7,0.915663,322430041,32243004106,0,PUNT,-3,1.0,1.915663,0.936364,UMass,UMass,Connecticut,UConn,1,70.0,2,1,False,UMass,0,1197.0,0.000000,False,False,False,0.337455,0.995614,0.658159
657100,322430041038,UMass,Mid-American,Connecticut,Big East,Connecticut,UMass,0,7,32243004103,1,19,57,75,2,6,0,Pass Incompletion,Mike Wegzyn pass incomplete to Michael Cox.,-0.539959,2012,322430041038,Connecticut,1,75,-7,1197,1.791759,False,False,7,0.915663,322430041,32243004106,0,PUNT,-3,1.0,1.915663,0.936364,UMass,UMass,Connecticut,UConn,1,75.0,3,6,False,UMass,0,1131.0,1.791759,False,False,False,-0.523452,-1.640028,-1.116575
657104,322430041042,UMass,Mid-American,Connecticut,Big East,Connecticut,UMass,0,7,32243004103,1,18,51,79,4,10,34,Punt,"Jeff Strait punt for 43 yards, returned by Nic...",NaN,2012,322430041042,Connecticut,1,79,-7,1131,2.302585,False,False,7,0.915663,322430041,32243004106,0,PUNT,-3,1.0,1.915663,0.936364,UMass,UMass,Connecticut,UConn,1,55.0,1,10,True,Connecticut,1,1125.0,2.302585,False,False,False,-1.270070,-1.782927,-0.512856
657105,322430041051,UMass,Mid-American,Connecticut,Big East,Connecticut,UMass,0,7,32243004105,1,16,55,63,1,10,-3,Pass Completion,Mike Wegzyn pass complete to Jordan Broadnax f...,-1.532323,2012,322430041051,Connecticut,1,63,-7,1015,2.302585,False,False,7,0.915663,322430041,32243004106,0,END OF GAME,-3,1.0,1.915663,0.936364,UMass,UMass,Connecticut,UConn,1,66.0,2,13,False,UMass,0,980.0,2.564949,False,False,False,1.082359,-0.679735,-1.762094
657106,322430041052,UMass,Mid-American,Connecticut,Big East,Connecticut,UMass,0,7,32243004105,1,16,20,66,2,13,3,Rush,Mike Wegzyn rush for 3 yards to the UMass 37.,-0.195592,2012,322430041052,Connecticut,1,66,-7,980,2.564949,False,False,7,0.915663,322430041,32243004106,0,END OF GAME,-3,1.0,1.915663,0.936364,UMass,UMass,Connecticut,UConn,1,63.0,3,10,False,UMass,0,900.0,2.302585,False,False,False,-0.679735,-0.946751,-0.267015
